---
title-block-banner: true
title: DCGAN face tutorial
subtitle: Generating Adversarial Example 
description: 실제 유명인들의 사진들로 적대적 생성 신경망(GAN)을 학습시켜, 새로운 유명인의 사진을 만든다.
categories:
    - PyTorch
    - computer vision
    - tutorial
date: 2024-03-10
author: dotpyo

toc: false      # 코드 영역때문에 페이지가 답답함
reference-location: document    # callout에서 사용하는 각주가 너무 많다.
---

## GAN?